In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools
import math
from torch.jit import script, trace
from torch import optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
corpus_name = 'chatData'
corpus = os.path.join(r'C:\Users\abc\jupyter\pytorch\chatbot', corpus_name)

def printLines(file, n=10):
    with open(file, 'r', encoding='utf-8') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)
        
printLines(os.path.join(corpus, 'ko_chatdata.txt'))

$

잘 마실게.

오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서..

무슨 소리야? 누군 약속 없어서 거기 가? 사람 증말..무조건 같이 가. 사과 하루이틀해? 진짜 어이없어.이거 안 마실래.

하..

$

언니 과외 마쳤어요? 오늘은 늦게까지 했네요.

기말고사라..내가 하나 들어줘요?

괜찮아요.

$



In [3]:
def loadLines(fileName):
    lines = []
    count = 0
    count_list = []
    result = []
    with open(fileName, 'r', encoding='utf-8') as f:
        next(f)
        for i, line in enumerate(f):
            line = line.strip()
            if line == '$':
                count_list.append(count)
                count = 0
                pass
            else:
                lines.append(line)
                count+= 1
        count_list.append(count)
        count = 0
        for i in count_list: # 6, 4, 2, 7
            for j in range(count+1, count+i):
                result.append(lines[j-1] + '\t' + lines[j])
            count += i
    return result
            
lines = loadLines(os.path.join(corpus, 'ko_chatdata.txt'))
print(lines[:5])

['잘 마실게.\t오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서..', '오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서..\t무슨 소리야? 누군 약속 없어서 거기 가? 사람 증말..무조건 같이 가. 사과 하루이틀해? 진짜 어이없어.이거 안 마실래.', '무슨 소리야? 누군 약속 없어서 거기 가? 사람 증말..무조건 같이 가. 사과 하루이틀해? 진짜 어이없어.이거 안 마실래.\t하..', '언니 과외 마쳤어요? 오늘은 늦게까지 했네요.\t기말고사라..내가 하나 들어줘요?', '기말고사라..내가 하나 들어줘요?\t괜찮아요.']


In [4]:
datafile = os.path.join(corpus, 'formatted_ko_conversations.txt')

delimiter = '\t'
delimiter = str(codecs.decode(delimiter, 'unicode_escape'))

print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in lines:
        writer.writerow([pair])
        
print("\nSample lines from file:")
printLines(datafile)


Writing newly formatted file...

Sample lines from file:
"잘 마실게.	오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서.."

"오후에 학부모 대표 모임에 사과하러 가는 거 교감선생님만 가시면 안돼요? 전 약속이 있어서..	무슨 소리야? 누군 약속 없어서 거기 가? 사람 증말..무조건 같이 가. 사과 하루이틀해? 진짜 어이없어.이거 안 마실래."

"무슨 소리야? 누군 약속 없어서 거기 가? 사람 증말..무조건 같이 가. 사과 하루이틀해? 진짜 어이없어.이거 안 마실래.	하.."

"언니 과외 마쳤어요? 오늘은 늦게까지 했네요.	기말고사라..내가 하나 들어줘요?"

"기말고사라..내가 하나 들어줘요?	괜찮아요."

"성적가지고 해리한테 너무 그러지 마. 애가 엄청 스트레스 받는 모양이던데. 아직 초등학생인데 점수 좀 못 받으면 뭐 어떻다고.	스트레스 받는 애 성적이 저 모양이야?"

"스트레스 받는 애 성적이 저 모양이야?	지도 잘하고 싶겠지. 지라고 못하고 싶겠어?"

"지도 잘하고 싶겠지. 지라고 못하고 싶겠어?	속상하잖아. 어떤 앤 산에서 살다 연필 잡은 지도 몇달 안됐는데 90점 100점 척척 받아오고 누군 해달라는 거 다 해주고 키웠는데 맨날 바닥이고."

"속상하잖아. 어떤 앤 산에서 살다 연필 잡은 지도 몇달 안됐는데 90점 100점 척척 받아오고 누군 해달라는 거 다 해주고 키웠는데 맨날 바닥이고.	또 비교한다. 당신 그렇게 비교하는 게 애한테 얼마나 안좋은."

"또 비교한다. 당신 그렇게 비교하는 게 애한테 얼마나 안좋은.	아 몰라! 비교가 안되야 비굘 안하지. 하여튼 맘에 안 들어. 나같음 자존심이 상해서라도 열심히 하겠는데..아우. 진짜 보고 있음 열불터져서.."



In [5]:
PAD_token = 0
SOS_token = 1
EOS_token = 2

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token:'PAD', SOS_token:'SOS', EOS_token:'EOS'}
        self.num_words = 3
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1
            
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        
        keep_words = []
        
        for k, v in self.word2count.items(): # k : 키값, v : 빈도수
            if v >= min_count:
                keep_words.append(k)
                
        print('keep_words {} / {} = {:.4f}'.format(len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)))
        
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token:'PAD', SOS_token:'SOS', EOS_token:'EOS'}
        self.num_words = 3
        
        for word in keep_words:
            self.addWord(word)

In [6]:
MAX_LENGTH = 10  

def readVocs(datafile, corpus_name): # corpus_name : chatData / datafile : formatted_ko_conversations.txt
    print('Reading lines...')
    lines = open(datafile, encoding='utf-8').read().strip().split('\n')
    pairs = [l.split('\t') for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs # voc : 문서 단어집합 / pairs : 문장 쌍 집합

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name) # voc : 단어집합, pairs : 질문 쌍
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)

for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 70560 sentence pairs
Trimmed to 41945 sentence pairs
Counting words...
Counted words: 64789
['"언니 과외 마쳤어요? 오늘은 늦게까지 했네요.', '기말고사라..내가 하나 들어줘요?"']
['"기말고사라..내가 하나 들어줘요?', '괜찮아요."']
['"스트레스 받는 애 성적이 저 모양이야?', '지도 잘하고 싶겠지. 지라고 못하고 싶겠어?"']
['"내가 보기엔', '나가서 냉수 한잔만 떠다줘. 아우. 괜히 또 열받네."']
['"나가서 냉수 한잔만 떠다줘. 아우. 괜히 또 열받네.', '내 생각엔"']
['"내 생각엔', '빨리 냉수 좀 떠줘."']
['"빨리 냉수 좀 떠줘.', '알았어."']
['"이순팔 맞는데..', '아깐 자기가 이순댄가 뭐그래놓구는.."']
['"아깐 자기가 이순댄가 뭐그래놓구는..', '이분 진짜 이름이 뭐예요?"']
['"이분 진짜 이름이 뭐예요?', '본인이 이순팔이라면 이순팔이겠죠 뭐. 전 가도 돼죠?"']


In [7]:
MIN_COUNT = 3

def trimRareWords(voc, pairs, MIN_COUNT):
    
    voc.trim(MIN_COUNT)
    
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        target_sentence = pair[1]
        keep_input = True
        keep_output = True
        
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
                
        for word in target_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break
                
        if keep_input and keep_output:
            keep_pairs.append(pair)
            
    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs

pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 21433 / 64786 = 0.3308
Trimmed from 41945 pairs to 8393, 0.2001 of total


In [27]:
def indexesFromSentence(voc, sentence):
    # print([voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]) : [8052, 20188, 9959, 20189, 16252, 2]
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l, fillvalue=PAD_token):
    # print(list(itertools.zip_longest(*l, fillvalue=fillvalue))) : (1957, 2, 2, 2, 0)
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l] 
    # print(indexes_batch) : [[5027, 1239, 9433, 2], [5951, 4686, 1476, 2], [1116, 5309, 2], [319, 2], [186, 2]]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch]) 
    # print(lengths) : tensor([4, 4, 3, 2, 2])
    padList = zeroPadding(indexes_batch)
    # print(padList) : [[5027, 1239, 9433, 2], [5951, 4686, 1476, 2], [1116, 5309, 2, 0], [319, 2, 0, 0], [186, 2, 0, 0]]
    padVar = torch.LongTensor(padList)
    return padVar, lengths

def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch]) 
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList) 
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len



small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
input_variable: tensor([[19428, 13031,   484,  7483,   101],
        [ 2260,   182,  8398,     2,     2],
        [  366,     2,     2,     0,     0],
        [    2,     0,     0,     0,     0]])
lengths: tensor([4, 3, 3, 2, 2])
target_variable: tensor([[ 1732,   798,  4726,  3475, 10261],
        [  434,  8792, 17215,  3476, 19650],
        [ 1642,     2, 12791,     2, 19651],
        [    2,     0, 17216,     0, 15394],
        [    0,     0, 13279,     0,   141],
        [    0,     0, 10224,     0,  5290],
        [    0,     0,     2,     0,     2]])
mask: tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 1, 0, 1],
        [0, 0, 1, 0, 1],
        [0, 0, 1, 0, 1],
        [0, 0, 1, 0, 1]], dtype=torch.uint8)
max_target_len: 7


In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, 
                          dropout = (0 if n_layers == 1 else dropout), bidirectional=True)
        
    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq) # input_seq : shape=(max_length, batch_size)
        # print(embedded.shape) : torch.Size([10, 64, 500]) [max_length, batch_size, hidden_size(은닉상태 크기)]
        
        # nn.utils.rnn.pack_padded_sequence : 패딩연산처리 쉽게하기 위해 중간에 빈공간 제거(형태 : tensor)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths) # input_lengths : shape=(batch_size)
        # print(packed.batch_sizes) : tensor([64, 64, 64, 58, 52, 45, 38, 17,  8,  2])
        
        outputs, hidden = self.gru(packed, hidden) # 입력hidden : shape=(n_layers * num_directions, batch_size, hidden_size)
        # print(outputs.batch_sizes) : tensor([64, 64, 63, 52, 47, 34, 24, 18, 12,  6])
        # print(hidden.shape) : torch.Size([4, 64, 500]) [층 * 양방향이면2 아니면1, batch_size, hidden_size]
        
        # nn.utils.rnn.pad_packed_sequence : 패딩연산이 끝난 것을 다시 원래대로 (형태 : torch)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # print(outputs.shape)# : torch.Size([10, 64, 1000]) # [max_length, batch_size, hidden_size(양방향으로 진행했으면 *2)]
        
        # 양방향 GRU의 출력을 합산합니다
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # print(outputs.shape) : torch.Size([10, 64, 500])
        
        # hidden : GRU의 최종 은닉 상태
        return outputs, hidden

In [10]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, 'is not an appropriate attention method.')
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))
            
    # 가중치 계산을 dot-product로 계산
    def dot_score(self, hidden, encoder_output):
        # print(torch.sum(hidden * encoder_output, dim=2).shape) : torch.Size([10, 64]) 10개 생성[max_length, batch_size]
        return torch.sum(hidden * encoder_output, dim=2)
    
    # 
    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        # print(energy.shape) : torch.Size([10, 64, 500]) 10개 생성[max_length, batch_size, hidden_size]
        
        # print(torch.sum(hidden * energy, dim=2).shape) : torch.Size([10, 64]) 10개 생성[max_length, batch_size]
        return torch.sum(hidden * energy, dim=2)
    
    
    def concat_score(self, hidden, encoder_output):
        # cat : 합칠 때 차원은 2차원으로 / expand : 확장
        # Tanh 함수는 함수값을 [-1, 1]로 제한시킴
        # print((hidden.expand(encoder_output.size(0), -1, -1).shape)) : torch.Size([10, 64, 500])
        # print(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2).shape) : torch.Size([10, 64, 1000])
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        # print(energy.shape) : torch.Size([10, 64, 500]) 10개 생성[max_length, batch_size, hidden_size]
        return torch.sum(self.v * energy, dim=2)
    
    def forward(self, hidden, encoder_outputs):
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)
            
        attn_energies = attn_energies.t() # t() : 행과 열을 바꿔서 저장[1, 2, 3], [4, 5, 6] -> [1, 4, 7], [2, 5, 8]
        
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [11]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # 참조를 보존해 둡니다
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # 레이어를 정의합니다
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # 주의: 한 단위 시간에 대해 한 단계(단어)만을 수행합니다
        # 현재의 입력 단어에 대한 임베딩을 구합니다   
        embedded = self.embedding(input_step) # input_step : 입력 시퀀스 배치에 대한 한 단위 시간(한 단어). shape=(1, batch_size)
        embedded = self.embedding_dropout(embedded)
        # print(embedded.shape) : torch.Size([1, 64, 500])
        
        # 양방향x
        # last_hidden : GRU의 마지막 은닉 레이어. shape=(n_layers * num_directions, batch_size, hidden_size)
        # print(last_hidden.shape) : torch.Size([2, 64, 500]) 
        rnn_output, hidden = self.gru(embedded, last_hidden) 
        # print(rnn_output.shape) : torch.Size([1, 64, 500])
        # print(hidden.shape) : torch.Size([2, 64, 500])

        # attention 가중치
        attn_weights = self.attn(rnn_output, encoder_outputs) # encoder_outputs : 인코더 모델 출력 shape=(max_length, batch_size, hidden_size)
        # print(attn_weights.shape) : torch.Size([64, 1, 10]) 

        # 인코더 출력에 어텐션을 곱하여 새로운 context vector생성
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # print(context.shape) : torch.Size([64, 1, 500])

        rnn_output = rnn_output.squeeze(0) # print(rnn_output.shape) : torch.Size([64, 500])
        context = context.squeeze(1) # print(context.shape) : torch.Size([64, 500])
        concat_input = torch.cat((rnn_output, context), 1) # print(concat_input.shape) : torch.Size([64, 1000])
        concat_output = torch.tanh(self.concat(concat_input))
        # print(concat_output.shape) : torch.Size([64, 500])

        # output : 각 단어가 디코딩된 시퀀스에서 다음 단어로 사용되었을 때 적합할 확률을 나타내는 정규화된 softmax 텐서. 
        # shape=(batch_size, voc.num_words)
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)

        return output, hidden

In [12]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [13]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding, encoder_optimizer, decoder_optimizer,
         batch_size, clip, max_length = MAX_LENGTH):
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    
    loss = 0
    print_losses = []
    n_totals = 0
    
    # EncoderRNN의 forward부분 실행
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
    
    # 초기 디코더 입력을 생성(각 문장을 SOS 토큰으로 시작)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)
    
    # 디코더의 초기 은닉 상태를 인코더의 마지막 은닉 상태로
    decoder_hidden = encoder_hidden[:decoder.n_layers]
    
    # teacher_forcing : Decoder부분에서 앞 단어가 잘못 추측되었을 경우 뒤에도 달라지니 정답을 입력해 주는 것
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        for t in range(max_target_len):
            # LuongAttnDecoderRNN의 forward로 실행
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
             # Teacher forcing 사용: 다음 입력을 현재의 목표로 둡니다
            decoder_input = target_variable[t].view(1, -1)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
            # Teacher forcing 미사용: 다음 입력을 디코더의 출력으로 둡니다
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
            
    loss.backward()
    
    # clip_grad_norm_: 그라디언트를 제자리에서 수정합니다
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return sum(print_losses) / n_totals

In [14]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # 각 단계에 대한 배치 설정
    # batch2TrainData : return inp, lengths, output, mask, max_target_len
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]


    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1


    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        
        input_variable, lengths, target_variable, mask, max_target_len = training_batch


        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        
        print_loss += loss


        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Checkpoint를 저장
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [15]:
# 탐욕적 디코딩(Greedy decoding) : 각 단계에 대해 단순히 decoder_output 에서 가장 높은 softmax값을 갖는 단어를 선택하는 방식
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):

        # EncoderRNN의 forward부분 실행
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)

        # encoder의 마지막 hidden이 decoder의 처음 hidden
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        
        # decoder의 처음입력을 SOS로 초기화
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token

        # 디코더가 단어를 덧붙여 나갈 텐서를 초기화
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)

        for _ in range(max_length):
            # LuongAttnDecoderRNN의 forward로 실행
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)

            # 가장 가능성 높은 단어 토큰과 그 softmax 점수를 구합니다
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)

            # 토큰, 점수 기록
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)

            # 현재의 토큰을 디코더의 다음 입력으로 준비시킵니다(차원을 증가시켜서)
            decoder_input = torch.unsqueeze(decoder_input, 0)

        return all_tokens, all_scores

In [16]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    # indexes_batch : 문장을 단어집합에 저장된 수로 바꾼후 마지막에 EOS추가하는 함수
    indexes_batch = [indexesFromSentence(voc, sentence)]
    
    # lengths 텐서를 만듭니다
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    
    # 배치의 차원을 뒤집어서 모델이 사용하는 형태로 만듭니다
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)

    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    
    # searcher를 이용하여 문장을 디코딩합니다
    tokens, scores = searcher(input_batch, lengths, max_length)
    
    # 인덱스 -> 단어
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # 입력 문장을 받아옵니다
            input_sentence = input('> ')
            # 종료 조건인지 검사합니다
            if input_sentence == 'q' or input_sentence == 'quit': break
            # 문장을 평가합니다
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # 응답 문장을 형식에 맞춰 출력합니다
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [17]:
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64


loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# loadFilename이 제공되는 경우에는 모델을 불러옵니다
if loadFilename:
    # 모델을 학습할 때와 같은 기기에서 불러오는 경우
    checkpoint = torch.load(loadFilename)
    # GPU에서 학습한 모델을 CPU로 불러오는 경우
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')

embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)

encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)

encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [18]:
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 2000
print_every = 1
save_every = 500

# Dropout 레이어를 학습 모드로 둡니다
encoder.train()
decoder.train()

# Optimizer를 초기화합니다
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)


for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# 학습 단계를 수행합니다
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...


<ipython-input-12-f319a975feb5>:4: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen/native/IndexingUtils.h:20.)
  loss = crossEntropy.masked_select(mask).mean()


Iteration: 1; Percent complete: 0.0%; Average loss: 9.9726
Iteration: 2; Percent complete: 0.1%; Average loss: 9.8936
Iteration: 3; Percent complete: 0.1%; Average loss: 9.8000
Iteration: 4; Percent complete: 0.1%; Average loss: 9.5860
Iteration: 5; Percent complete: 0.1%; Average loss: 9.2851
Iteration: 6; Percent complete: 0.1%; Average loss: 8.9995
Iteration: 7; Percent complete: 0.2%; Average loss: 8.6445
Iteration: 8; Percent complete: 0.2%; Average loss: 8.0863
Iteration: 9; Percent complete: 0.2%; Average loss: 7.8818
Iteration: 10; Percent complete: 0.2%; Average loss: 8.3809
Iteration: 11; Percent complete: 0.3%; Average loss: 8.5496
Iteration: 12; Percent complete: 0.3%; Average loss: 8.7125
Iteration: 13; Percent complete: 0.3%; Average loss: 8.9074
Iteration: 14; Percent complete: 0.4%; Average loss: 8.4809
Iteration: 15; Percent complete: 0.4%; Average loss: 7.5465
Iteration: 16; Percent complete: 0.4%; Average loss: 7.8928
Iteration: 17; Percent complete: 0.4%; Average lo

Iteration: 139; Percent complete: 3.5%; Average loss: 6.6816
Iteration: 140; Percent complete: 3.5%; Average loss: 6.6842
Iteration: 141; Percent complete: 3.5%; Average loss: 6.7561
Iteration: 142; Percent complete: 3.5%; Average loss: 6.5542
Iteration: 143; Percent complete: 3.6%; Average loss: 6.6598
Iteration: 144; Percent complete: 3.6%; Average loss: 6.5898
Iteration: 145; Percent complete: 3.6%; Average loss: 6.7448
Iteration: 146; Percent complete: 3.6%; Average loss: 6.9520
Iteration: 147; Percent complete: 3.7%; Average loss: 6.5908
Iteration: 148; Percent complete: 3.7%; Average loss: 6.7723
Iteration: 149; Percent complete: 3.7%; Average loss: 6.8603
Iteration: 150; Percent complete: 3.8%; Average loss: 6.5269
Iteration: 151; Percent complete: 3.8%; Average loss: 6.4539
Iteration: 152; Percent complete: 3.8%; Average loss: 6.5811
Iteration: 153; Percent complete: 3.8%; Average loss: 6.6585
Iteration: 154; Percent complete: 3.9%; Average loss: 6.6077
Iteration: 155; Percent 

Iteration: 275; Percent complete: 6.9%; Average loss: 6.2573
Iteration: 276; Percent complete: 6.9%; Average loss: 6.3327
Iteration: 277; Percent complete: 6.9%; Average loss: 5.7585
Iteration: 278; Percent complete: 7.0%; Average loss: 6.1682
Iteration: 279; Percent complete: 7.0%; Average loss: 5.8598
Iteration: 280; Percent complete: 7.0%; Average loss: 5.9333
Iteration: 281; Percent complete: 7.0%; Average loss: 5.9334
Iteration: 282; Percent complete: 7.0%; Average loss: 6.0929
Iteration: 283; Percent complete: 7.1%; Average loss: 5.6278
Iteration: 284; Percent complete: 7.1%; Average loss: 5.8932
Iteration: 285; Percent complete: 7.1%; Average loss: 6.0542
Iteration: 286; Percent complete: 7.1%; Average loss: 5.9045
Iteration: 287; Percent complete: 7.2%; Average loss: 5.9362
Iteration: 288; Percent complete: 7.2%; Average loss: 6.1580
Iteration: 289; Percent complete: 7.2%; Average loss: 5.9768
Iteration: 290; Percent complete: 7.2%; Average loss: 6.0941
Iteration: 291; Percent 

...

Iteration: 3454; Percent complete: 86.4%; Average loss: 0.1596
Iteration: 3455; Percent complete: 86.4%; Average loss: 0.2658
Iteration: 3456; Percent complete: 86.4%; Average loss: 0.2343
Iteration: 3457; Percent complete: 86.4%; Average loss: 0.1967
Iteration: 3458; Percent complete: 86.5%; Average loss: 0.2133
Iteration: 3459; Percent complete: 86.5%; Average loss: 0.2162
Iteration: 3460; Percent complete: 86.5%; Average loss: 0.2284
Iteration: 3461; Percent complete: 86.5%; Average loss: 0.2419
Iteration: 3462; Percent complete: 86.6%; Average loss: 0.1636
Iteration: 3463; Percent complete: 86.6%; Average loss: 0.2335
Iteration: 3464; Percent complete: 86.6%; Average loss: 0.2568
Iteration: 3465; Percent complete: 86.6%; Average loss: 0.1989
Iteration: 3466; Percent complete: 86.7%; Average loss: 0.2552
Iteration: 3467; Percent complete: 86.7%; Average loss: 0.2771
Iteration: 3468; Percent complete: 86.7%; Average loss: 0.2382
Iteration: 3469; Percent complete: 86.7%; Average loss:

Iteration: 3586; Percent complete: 89.6%; Average loss: 0.1067
Iteration: 3587; Percent complete: 89.7%; Average loss: 0.2039
Iteration: 3588; Percent complete: 89.7%; Average loss: 0.2588
Iteration: 3589; Percent complete: 89.7%; Average loss: 0.2259
Iteration: 3590; Percent complete: 89.8%; Average loss: 0.2575
Iteration: 3591; Percent complete: 89.8%; Average loss: 0.1372
Iteration: 3592; Percent complete: 89.8%; Average loss: 0.1542
Iteration: 3593; Percent complete: 89.8%; Average loss: 0.2323
Iteration: 3594; Percent complete: 89.8%; Average loss: 0.3190
Iteration: 3595; Percent complete: 89.9%; Average loss: 0.2000
Iteration: 3596; Percent complete: 89.9%; Average loss: 0.1897
Iteration: 3597; Percent complete: 89.9%; Average loss: 0.1762
Iteration: 3598; Percent complete: 90.0%; Average loss: 0.1787
Iteration: 3599; Percent complete: 90.0%; Average loss: 0.1478
Iteration: 3600; Percent complete: 90.0%; Average loss: 0.1446
Iteration: 3601; Percent complete: 90.0%; Average loss:

Iteration: 3718; Percent complete: 93.0%; Average loss: 0.1552
Iteration: 3719; Percent complete: 93.0%; Average loss: 0.2164
Iteration: 3720; Percent complete: 93.0%; Average loss: 0.2453
Iteration: 3721; Percent complete: 93.0%; Average loss: 0.2944
Iteration: 3722; Percent complete: 93.0%; Average loss: 0.2976
Iteration: 3723; Percent complete: 93.1%; Average loss: 0.3297
Iteration: 3724; Percent complete: 93.1%; Average loss: 0.2907
Iteration: 3725; Percent complete: 93.1%; Average loss: 0.3240
Iteration: 3726; Percent complete: 93.2%; Average loss: 0.3065
Iteration: 3727; Percent complete: 93.2%; Average loss: 0.1777
Iteration: 3728; Percent complete: 93.2%; Average loss: 0.2482
Iteration: 3729; Percent complete: 93.2%; Average loss: 0.2934
Iteration: 3730; Percent complete: 93.2%; Average loss: 0.2908
Iteration: 3731; Percent complete: 93.3%; Average loss: 0.2342
Iteration: 3732; Percent complete: 93.3%; Average loss: 0.2070
Iteration: 3733; Percent complete: 93.3%; Average loss:

Iteration: 3850; Percent complete: 96.2%; Average loss: 0.2959
Iteration: 3851; Percent complete: 96.3%; Average loss: 0.2943
Iteration: 3852; Percent complete: 96.3%; Average loss: 0.2259
Iteration: 3853; Percent complete: 96.3%; Average loss: 0.2498
Iteration: 3854; Percent complete: 96.4%; Average loss: 0.2674
Iteration: 3855; Percent complete: 96.4%; Average loss: 0.2244
Iteration: 3856; Percent complete: 96.4%; Average loss: 0.2799
Iteration: 3857; Percent complete: 96.4%; Average loss: 0.2470
Iteration: 3858; Percent complete: 96.5%; Average loss: 0.2558
Iteration: 3859; Percent complete: 96.5%; Average loss: 0.3170
Iteration: 3860; Percent complete: 96.5%; Average loss: 0.1886
Iteration: 3861; Percent complete: 96.5%; Average loss: 0.2561
Iteration: 3862; Percent complete: 96.5%; Average loss: 0.2293
Iteration: 3863; Percent complete: 96.6%; Average loss: 0.3156
Iteration: 3864; Percent complete: 96.6%; Average loss: 0.2154
Iteration: 3865; Percent complete: 96.6%; Average loss:

Iteration: 3981; Percent complete: 99.5%; Average loss: 0.1883
Iteration: 3982; Percent complete: 99.6%; Average loss: 0.1923
Iteration: 3983; Percent complete: 99.6%; Average loss: 0.1300
Iteration: 3984; Percent complete: 99.6%; Average loss: 0.1170
Iteration: 3985; Percent complete: 99.6%; Average loss: 0.1956
Iteration: 3986; Percent complete: 99.7%; Average loss: 0.2830
Iteration: 3987; Percent complete: 99.7%; Average loss: 0.3989
Iteration: 3988; Percent complete: 99.7%; Average loss: 0.3097
Iteration: 3989; Percent complete: 99.7%; Average loss: 0.1843
Iteration: 3990; Percent complete: 99.8%; Average loss: 0.1525
Iteration: 3991; Percent complete: 99.8%; Average loss: 0.2776
Iteration: 3992; Percent complete: 99.8%; Average loss: 0.2318
Iteration: 3993; Percent complete: 99.8%; Average loss: 0.2017
Iteration: 3994; Percent complete: 99.9%; Average loss: 0.2560
Iteration: 3995; Percent complete: 99.9%; Average loss: 0.3795
Iteration: 3996; Percent complete: 99.9%; Average loss:

In [19]:
# Dropout 레이어를 평가 모드로 설정합니다
encoder.eval()
decoder.eval()

# 탐색 모듈을 초기화합니다
searcher = GreedySearchDecoder(encoder, decoder)

# 채팅을 시작합니다 (다음 줄의 주석을 제거하면 시작해볼 수 있습니다)
evaluateInput(encoder, decoder, searcher, voc)

> 안녕
Bot: 네?"
> 안녕
Bot: 네?"
> 잘있어
Error: Encountered unknown word.
> 뭐하고 지내?
Error: Encountered unknown word.
> 뭐해?
Bot: 어.
> 어?
Bot: 무슨 일 있었어?"
> 학습
Error: Encountered unknown word.
> 학습 개똥이네
Error: Encountered unknown word.
> 왜그러냐고
Error: Encountered unknown word.
> 왜
Bot: 어디 데릴러 다 물어보면 될까요?"
> 왜
Bot: 어디 데릴러 다 물어보면 될까요?"
> 화나네
Error: Encountered unknown word.
> 김치
Error: Encountered unknown word.
> 나한테
Bot: 어떻게 어떻게 어떻게 아니야."
> ㅇ?
Error: Encountered unknown word.
> 응?
Bot: 왜 그러세요?"
> 응?
Bot: 왜 그러세요?"
> 아?
Error: Encountered unknown word.
> 어?
Bot: 무슨 일 있었어?"
> 후
Bot: 하.."
> 그럼
Bot: 왜?"
> 왜자꾸 왜야
Error: Encountered unknown word.
> 왜야
Error: Encountered unknown word.
> 스트레스 받아
Bot: 넌 잘 갈려고?"
> 기말고사끝났어?
Error: Encountered unknown word.
> 기말고사
Error: Encountered unknown word.
> 기말고사라
Error: Encountered unknown word.
> 괜찮아요
Error: Encountered unknown word.
> 괜찮요
Error: Encountered unknown word.
> 괜찮네요
Error: Encountered unknown word.
> 후
Bot: 하.."
> 푸
Error: Encountered unknown word.
> 자옥씨
